In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import os, sys
sys.path.insert(0, os.path.abspath(".."))
from utilidades.calibration import utilities as ult

import sklearn as sk
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import optuna 
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score
import time
import skopt
from skopt import BayesSearchCV

import sklearn.pipeline
from sklearn.ensemble import RandomForestClassifier
from lightgbm import early_stopping


(DONE) * Lightgbm       - Algorítimico 
(DONE) * Smote          - Preprocessing

- Vini
* CDBH           - Híbrido - Mandar email
* OSM Classifier - Algorítimico - Mandar email

(DONE)* SVM VR         - Preprocessing 98%. Testando a Classe

(STARTING)* DBSMOTE IN R      - Preprocessing Construir no R. 0%


- Manaus
(DOING)* HDDT Emsemble  - Algorítimico Implementado no R 0%
(DOING)* CCR            - Preprocessing 15%

(TO DO) POTENCIAL ANCHORING      - Preprocessing

(DONE) * EasyEmsemble   - Algorítimico
(DONE)* Método novo    - Algorítimico


#### CANCELED's

(CANCELED)* EHSO           - Preprocessing
(CANCELED)* WHMBoost       - Cost Sensitivity EMAIL ENVIADO
(CANCELED)* CHMDT          - Cost Sensitivity EMAIL ENVIADO

https://www.rdocumentation.org/packages/smotefamily/versions/1.3.1/topics/DBSMOTE

In [ ]:
# X_ = df.drop('diagnosis', axis=1)
# y = df[['diagnosis']]

# X = np.asarray(X_)
# y = np.asarray(y).ravel()

# X_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=32,stratify=y)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)


# SVM_vr_search = BayesSearchCV(
#     SVC(),
#     {
#         'C': Integer(5,10),
#         'gamma': Integer(5,10),
#         'degree': Integer(1,8),
#         'kernel': Categorical(['linear', 'poly', 'rbf','sigmoid'])
#     },
#      n_iter=32,
#      random_state=32,
#      cv=5,
#      scoring='accuracy',
#      return_train_score=True
# )

# SVM_vr_search.fit(X_train_scaled, y_train)

# X_test_scaled = scaler.transform(x_test)
# print(SVM_vr_search.score(X_test_scaled, y_test))

# results_cv = pd.DataFrame(SVM_vr_search.cv_results_)

# temp = results_cv[['mean_train_score', 'mean_test_score']]
# temp['diff'] = temp['mean_test_score'] - temp['mean_train_score']
# to_go = temp[abs(temp['diff']) < 0.05].sort_values(by = 'mean_test_score', ascending = False).head(1).index

# params = results_cv.loc[to_go.values[0]]
# kwargs = params.params
# print(kwargs)

# best_SVM = SVC(random_state=32,**kwargs).fit(X_train_scaled,y_train)
# y_train2= best_SVM.predict(X_train_scaled)
# X_testz = scaler.inverse_transform(X_train_scaled)
# new_train = pd.DataFrame(X_testz,columns=X_.columns)
# new_train['y'] = y_train2

In [2]:
df = pd.read_csv('breast-cancer.csv') ; print(df.shape,df.select_dtypes('object').value_counts())

(569, 32) diagnosis
B            357
M            212
dtype: int64


In [3]:
num_cols = df.select_dtypes(include=[float, int]).columns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

if num_cols.values.shape[0] > 0:
            pipe_num = Pipeline(
            steps = [
                ("selector_num", ColumnTransformer([("selector", "passthrough", num_cols.values)], remainder = 'drop')),
                ('num_imputer', SimpleImputer(strategy='mean'))])

In [4]:
try:
    df.drop('id',axis=1, inplace=True)
except:
    pass
df.diagnosis.replace('B',0,inplace=True) # Benignos para 0
df.diagnosis.replace('M',1,inplace=True) # Malignos para 1

In [ ]:
import skopt
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler


X_ = df.drop('diagnosis', axis=1)
y = df[['diagnosis']]

X = np.asarray(X_)
y = np.asarray(y).ravel()

X_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=32,stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


SVM_vr_search = BayesSearchCV(
    SVC(),
    {
        'C': Integer(5,10),
        'gamma': Integer(5,10),
        'degree': Integer(1,8),
        'kernel': Categorical(['linear', 'poly', 'rbf','sigmoid'])
    },
     n_iter=32,
     random_state=32,
     cv=5,
     scoring='accuracy',
     return_train_score=True
)

SVM_vr_search.fit(X_train_scaled, y_train)

X_test_scaled = scaler.transform(x_test)
print(SVM_vr_search.score(X_test_scaled, y_test))

results_cv = pd.DataFrame(SVM_vr_search.cv_results_)

temp = results_cv[['mean_train_score', 'mean_test_score']]
temp['diff'] = temp['mean_test_score'] - temp['mean_train_score']
to_go = temp[abs(temp['diff']) < 0.05].sort_values(by = 'mean_test_score', ascending = False).head(1).index

params = results_cv.loc[to_go.values[0]]
kwargs = params.params
print(kwargs)

best_SVM = SVC(random_state=32,**kwargs).fit(X_train_scaled,y_train)
y_train2= best_SVM.predict(X_train_scaled)
X_testz = scaler.inverse_transform(X_train_scaled)
new_train = pd.DataFrame(X_testz,columns=X_.columns)
new_train['y'] = y_train2

c:\Users\vinic\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\vinic\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\vinic\anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


0.9627659574468085
OrderedDict([('C', 5), ('degree', 1), ('gamma', 10), ('kernel', 'linear')])


C:\Users\vinic\AppData\Local\Temp/ipykernel_12368/1773425698.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['diff'] = temp['mean_test_score'] - temp['mean_train_score']


In [ ]:
X,y = ult.splitxy(df,'diagnosis')
X_train, y_train, X_test, y_test, X_val, y_val = ult.train_test_val(X,y)

prep_feat_tuple = ult.create_prep_pipe(df,'diagnosis')
prep_feat = prep_feat_tuple[0]

lists_pandarizer = list(prep_feat_tuple[1]) + list(prep_feat_tuple[2])

pipe_prep = Pipeline([
            ('transformer_prep', prep_feat),
            ("pandarizer", FunctionTransformer(lambda x: pd.DataFrame(x, columns = lists_pandarizer))),
        ])
pipe_prep.fit(X_train)

#X_train = np.asarray(X_train)

SVC(random_state=42)

metric = 'accuracy'

fit_params = {
    'eval_metric': metric, 
    'eval_set': [(X_test, pd.DataFrame(y_test))],
}

pipe_tuning = Pipeline([
    ('transformer_prep', prep_feat),
    ("pandarizer", FunctionTransformer(lambda x: pd.DataFrame(x, columns = lists_pandarizer))),
    ('estimator', SVC())
])

SVC_search_space = {
    "estimator__C": Integer(5, 10),   
    "estimator__gamma": Integer(5, 10),
    "estimator__degree": Integer(1,8),
    "estimator__kernel": Categorical(['linear', 'poly', 'rbf','sigmoid'])
}   

SVM_vr_search = BayesSearchCV(
    pipe_tuning,
    SVC_search_space,   
    n_iter=2,
    random_state=42,
    cv=5,
    scoring=metric,
    return_train_score=True,
    fit_params = fit_params,
)
SVM_vr_search.fit(X_train, y_train)

In [14]:
import skopt
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

In [18]:
class svm_vr():
    def __init__(self, dataframe : pd.DataFrame, target: str,metric : str = 'accuracy', pipe_final : sklearn.pipeline = None):
        self.dataframe = dataframe
        self.target = target
        self.metric = metric
        self._pipe_final = pipe_final

    def fit(self,random_state=42):
        # return y_train2 é o retorno dessa função. e utilizo como resposta
        # Splito os dados aqui

        X,y = ult.splitxy(self.dataframe,self.target)

        self.X_train, self.y_train, self.X_test, self.y_test, self.X_val, self.y_val = ult.train_test_val(X,y)

        prep_feat_tuple = ult.create_prep_pipe2(self.dataframe,self.target)
        self.prep_feat = prep_feat_tuple[0]

        lists_pandarizer = list(prep_feat_tuple[1]) + list(prep_feat_tuple[2])

        self.pipe_prep = Pipeline([
                    ('transformer_prep', self.prep_feat),
                    ("pandarizer", FunctionTransformer(lambda x: pd.DataFrame(x, columns = lists_pandarizer))),
                ])

        print(self.X_train)
        # self.pipe_prep.fit(self.X_train) # Precisa do self?

        y_train2 = svm_vr()

        LGBM = LGBMClassifier(random_state = 42, n_jobs = -1)

        pipe_tuning = Pipeline([
            ('transformer_prep', self.prep_feat),
            ("pandarizer", FunctionTransformer(lambda x: pd.DataFrame(x, columns = lists_pandarizer))),
            ('estimator', LGBM)
        ])
        
        self.pipe_prep = Pipeline([
            ('transformer_prep', self.prep_feat),
            ("pandarizer", FunctionTransformer(lambda x: pd.DataFrame(x, columns = lists_pandarizer))),
        ])
        self.pipe_prep.fit(X_train)       
        
        cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.3, random_state = 42)
        
        metric = self.metric
        
        fit_params = {
            'eval_metric': metric, 
            'eval_set': [(self.X_test, pd.DataFrame(self.y_test))],
            'callbacks': [(early_stopping(stopping_rounds = 10, verbose = True))],
        }        
        
        LGBM_search_space = {
            "estimator__learning_rate": Real(0.001, 0.01, prior = 'log-uniform'),
            "estimator__n_estimators": Integer(100, 1000),
            "estimator__class_weight": Categorical(['balanced', None]),
            "estimator__num_leaves": Integer(32, 256),
            "estimator__min_child_samples": Integer(100, 1000),
            "estimator__reg_alpha": Real(0, 100, prior = 'uniform'),
            "estimator__reg_lambda": Real(10., 200., prior = 'uniform'),
            "estimator__objective": Categorical(['binary']),
            "estimator__importance_type":Categorical(['gain']),
            "estimator__boosting_type": Categorical(['goss'])
        }    
        
        LGBM_bayes_search = BayesSearchCV(pipe_tuning, LGBM_search_space, n_iter = 2, scoring = metric, 
                                         return_train_score = True, 
                                         fit_params = fit_params,
                                         n_jobs = -1, cv = cv, random_state = 42, optimizer_kwargs = {'base_estimator': 'GP'})
        
        
        LGBM_bayes_search.fit(self.X_train, self.y_train2)        
        
        results_cv = pd.DataFrame(LGBM_bayes_search.cv_results_)
        
        temp = results_cv[['mean_train_score', 'mean_test_score']]
        temp['diff'] = temp['mean_test_score'] - temp['mean_train_score']
        to_go = temp[abs(temp['diff']) < 0.05].sort_values(by = 'mean_test_score', ascending = False).head(1).index
        
        params = results_cv.loc[to_go.values[0]]
        kwargs = params.params   
        print(kwargs)
        
        best_LGBM = LGBMClassifier(random_state = 42, n_jobs = -1, verbose = -1, **kwargs)
        
        best_LGBM.fit(self.pipe_prep.transform(self.X_train), self.y_train, early_stopping_rounds = 10, verbose = 20, eval_metric = metric,
                     eval_set = [(self.pipe_prep.transform(self.X_test), self.y_test)]) 
        
        
        pipe_final = Pipeline(
        [
            ('pipe_transformer_prep', self.pipe_prep),
            ('pipe_estimator', best_LGBM)
        ])       
        
        self._pipe_final = pipe_final

    def svm_vr(self,random_state=42):
        """ This function fits a SVM Classifier, taking the best result from the grid searching.
        After the best fit has been achieved, the training dependent variable values must be replaced by the predicts of the trained model, thus obtaining the balancing.

        Following the instructions of the Authors.
        They suggest using higher values for C and the RBF kernel function.

        "The proposed methodology followed a two phase
        approach. During the first phase the available training data is used
        to train SVM. Later, the target values of the training data are replaced
        by the corresponding predictions of the trained SVM. During the
        second phase, this modified data is used to train MLP, LR, and RF
        separately" Authors.

        Based on the ideia by the authors, we select the best model, by the value of accuracy
        """
        metric = self.metric

        SVM = SVC(random_state=42)

        pipe_tuning = Pipeline([
            ('transformer_prep', self.prep_feat),
            ("pandarizer", FunctionTransformer(lambda x: pd.DataFrame(x, columns = lists_pandarizer))),
            ('estimator', SVM)
        ])

        SVC_search_space = {
            "estimator__C": Integer(5, 10),   
            "estimator__gamma": Integer(5, 10),
            "estimator__degree": Integer(1,8),
            "estimator__kernel": Categorical(['linear', 'poly', 'rbf','sigmoid'])
        }   

        SVM_vr_search = BayesSearchCV(
            pipe_tuning,
            SVC_search_space,   
            n_iter=2,
            random_state=42,
            cv=5,
            scoring=metric,
            return_train_score=True,
        )
        SVM_vr_search.fit(self.X_train, self.y_train)

        results_cv = pd.DataFrame(SVM_vr_search.cv_results_)
        
        temp = results_cv[['mean_train_score', 'mean_test_score']]
        temp['diff'] = temp['mean_test_score'] - temp['mean_train_score']
        to_go = temp[abs(temp['diff']) < 0.05].sort_values(by = 'mean_test_score', ascending = False).head(1).index
        
        params = results_cv.loc[to_go.values[0]]
        kwargs = params.params
        print(kwargs)

        best_SVM = SVC(random_state=random_state,**kwargs).fit(X_train,y_train)
        y_train2= best_SVM.predict(X_train)
        return y_train2


In [19]:
svm_testing = svm_vr(df, 'diagnosis')

In [20]:
svm_testing.fit()

     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
473        12.27         29.97           77.42      465.4          0.07699   
89         14.64         15.24           95.77      651.9          0.11320   
566        16.60         28.08          108.30      858.1          0.08455   
189        12.30         15.90           78.83      463.7          0.08080   
493        12.46         12.83           78.83      477.3          0.07372   
..           ...           ...             ...        ...              ...   
166        10.80          9.71           68.77      357.6          0.09594   
332        11.22         19.86           71.94      387.3          0.10540   
465        13.24         20.13           86.87      542.9          0.08284   
335        17.06         21.00          111.80      918.6          0.11190   
312        12.76         13.37           82.29      504.1          0.08794   

     compactness_mean  concavity_mean  concave points_mean  sym

c:\Users\vinic\Documents\Projetos\TCC\utilidades\calibration.py:86: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  cat_cols = dataframe.select_dtypes(include=[object, pd.datetime]).columns


TypeError: __init__() missing 2 required positional arguments: 'dataframe' and 'target'

#### Generalizar as variavéis caso apareça no X variáveis categoricas CHECK
#### Entender o pipe do Manaus - Para esse tipo de variáveis. CHECK

#### SMOTE

#### Treino Teste a Val. CHECK

- Preprocesso Treino 
- Teste para hiperparâmetro.
- Validação checo a performance do modelo


#### Adicionar o fit_params no BayesSearchCV
#### Adicionar o RF classifier (Predict e validação final) sem Tunning CHECK